In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE273630"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE273630"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE273630.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE273630.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE273630.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dopamine-regulated biomarkers in peripheral blood of HIV+ Methamphetamine users"
!Series_summary	"HIV and Methamphetamine study - Translational Methamphetamine AIDS Research Center - Dopamine-regulated inflammatory biomarkers"
!Series_summary	"A digital transcript panel was custom-made based on Hs_NeuroPath_v1 (Nanostring) to accommodate dopamine-regulated inflammatory genes that were previously identified in vitro, and hypothesized to cluster HIV+ Methamphetamine users."
!Series_overall_design	"Specimens were peripheral blood leukocytes isolated from participants that included adults enrolled by NIH-funded studies at the University of California San Diego’s HIV Neurobehavioral Research Program (HNRP) and Translational Methamphetamine Research Center (TMARC) under informed consent and approved protocols. The subset of PWH and PWoH selected for this study were by design males, between 35 – 44 years old, due to cohort characteristics and to increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Dict, Any, List, Optional, Callable
import json

# 1. Determine if this dataset contains gene expression data
# Based on the background information, this appears to be a study of gene expression in peripheral blood leukocytes
# The Series summary mentions "dopamine-regulated inflammatory biomarkers" and a "digital transcript panel"
# This indicates gene expression data is likely present
is_gene_available = True

# 2. Variable availability and data type conversion
# Analyzing the sample characteristics dictionary
# Looking at the provided dictionary, we don't see explicit trait (Depression), age, or gender variables
# However, the overall design indicates all participants are males between 35-44 years old
# The study focuses on HIV status and Methamphetamine use, not depression

# 2.1 Data Availability
# No depression trait data is available
trait_row = None

# Age is not available as a variable (all participants are 35-44 years old per design)
age_row = None

# Gender is not available as a variable (all participants are males per design)
gender_row = None

# 2.2 Data Type Conversion
# Since trait data (Depression) is not available, we define a placeholder function
def convert_trait(value):
    return None

# Age conversion function (not used but defined for completeness)
def convert_age(value):
    return None

# Gender conversion function (not used but defined for completeness)
def convert_gender(value):
    return None

# 3. Save Metadata
# Validate and save the cohort information
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None, meaning clinical data for our trait is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE273630/GSE273630_series_matrix.txt.gz
Gene data shape: (780, 99)
First 20 gene/probe identifiers:
Index(['ABAT', 'ABL1', 'ACAA1', 'ACHE', 'ACIN1', 'ACTN1', 'ACVRL1', 'ADAM10',
       'ADCY5', 'ADCY8', 'ADCY9', 'ADCYAP1', 'ADORA1', 'ADORA2A', 'ADRA2A',
       'ADRB2', 'AGER', 'AIF1', 'AKT1', 'AKT1S1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be standard human gene symbols (like ABAT, ABL1, etc.)
# They match official HGNC gene symbols, so no mapping is necessary

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Since requires_gene_mapping is False (from step 4), we can use the gene data directly
    # Still need to normalize the gene symbols using the NCBI Gene synonym information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # 2-6. Skip linking clinical and genetic data because trait data is not available
    # This was determined in Step 2 (is_trait_available = False)
    print("Skipping clinical-genetic data linking as trait data is not available")
    
    # Use is_final=False since we can't provide required parameters for final validation
    is_usable = validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        note="Dataset contains gene expression data but lacks information about depression. The study focuses on HIV status and methamphetamine use in male participants aged 35-44."
    )
    
    print(f"Dataset usability for trait association studies: {is_usable}")
    
except Exception as e:
    print(f"Error processing gene data: {e}")
    # Record that this dataset is not usable
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False,
        note=f"Error processing gene data: {str(e)}"
    )

Normalized gene data shape: (780, 99)
Normalized gene data saved to ../../output/preprocess/Depression/gene_data/GSE273630.csv
Skipping clinical-genetic data linking as trait data is not available
Dataset usability for trait association studies: False
